In [1]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)

import re
import os
import time
import json
import numpy as np
import pandas as pd
import pickle as pkl
from tensorflow import keras
import matplotlib.pyplot as plt
from collections import defaultdict 
import tensorflow.keras.backend as K

from tensorflow import one_hot
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import TimeDistributed

from tensorflow.keras.optimizers import Adam

TEST = 50
EPOCHS = 15
LIMIT = 700
MAX_LEN = 50
BATCH_SIZE = 16
HIDDEN_DIM = 100
VECTOR_DIM = 100
VALID_SPLIT = 0.15

def get_data(path, setNum):

    X = []
    Y = []

    data = json.load(open(path))

    if setNum:
        data = data[LIMIT*(setNum-1):LIMIT*setNum] 

    for dict in data:

        x = dict['input']
        y = dict['output']

        n = len(x)

        for i in range(n):
            x[i] = re.sub('@[^ ]+','<username>',x[i])
            x[i] = re.sub('http://[^ ]+','<link>',x[i])

            y[i] = re.sub('@[^ ]+','<username>',y[i])
            y[i] = re.sub('http://[^ ]+','<link>',y[i])

        for _ in range(MAX_LEN-n):
            x.append("")
            y.append("")
        
        X.append(x)
        Y.append(y)

    return X, Y

def buildDict(data):

    wordToNum = defaultdict(int)
    num = 1
    for sent in data:
        for word in sent:
            if not wordToNum[word]:
                wordToNum[word] = num
                num+= 1

    return wordToNum

def buildDictInv(wordToNum):

    numToWord = defaultdict(str)

    for key in wordToNum.keys():
        numToWord[wordToNum[key]]=key

    return numToWord

def tokenize(data,wordToNum):

    tokenizedData = []

    for sent in data:

        tokenizedSent = []
        for word in sent:
            tokenizedSent.append(wordToNum[word])

        tokenizedSent=np.array(tokenizedSent,dtype=float)
        tokenizedData.append(tokenizedSent)

    return np.array(tokenizedData)

def getModel(VOCAB_SIZE):

    model = Sequential()

    model.add(Input(shape=(MAX_LEN,)))
    model.add(Embedding(VOCAB_SIZE, output_dim=VECTOR_DIM, input_length=MAX_LEN, trainable=True))
    model.add(LSTM(HIDDEN_DIM, return_sequences = True))
    model.add(TimeDistributed(Dense(VOCAB_SIZE)))
    model.add(Activation('softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    print(model.summary())
    
    return model

def defineModel():

    raw, normalized = get_data('data.json', 0)

    wordToNum = buildDict(raw + normalized)
    numToWord = buildDictInv(wordToNum)
    vocab_size = len(wordToNum)+1

    model       = getModel(vocab_size)

    model.save('model.h5')

    return wordToNum, numToWord

wordToNum, numToWord = defineModel()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 100)           1634100   
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           80400     
_________________________________________________________________
time_distributed (TimeDistri (None, 50, 16341)         1650441   
_________________________________________________________________
activation (Activation)      (None, 50, 16341)         0         
Total params: 3,364,941
Trainable params: 3,364,941
Non-trainable params: 0
_________________________________________________________________
None


In [2]:
def train(setNum):

  model = keras.models.load_model('model.h5')

  raw, normalized = get_data('data.json', setNum)

  raw = tokenize(raw, wordToNum)
  normalized = tokenize(normalized, wordToNum)

  x = raw
  y = normalized
  y = one_hot(y,len(wordToNum)+1, on_value=1, off_value=0)

  model.fit(x, y, batch_size = BATCH_SIZE, epochs = EPOCHS, validation_split = VALID_SPLIT)

  model.save('model.h5')

In [3]:
train(1)

Epoch 1/15
38/38 [==============================] - 35s 95ms/step - loss: 7.8208 - accuracy: 0.6779 - val_loss: 4.2511 - val_accuracy: 0.7211
Epoch 2/15
38/38 [==============================] - 3s 74ms/step - loss: 3.0979 - accuracy: 0.6928 - val_loss: 2.7039 - val_accuracy: 0.7211
Epoch 3/15
38/38 [==============================] - 3s 75ms/step - loss: 2.7273 - accuracy: 0.6928 - val_loss: 2.7026 - val_accuracy: 0.7211
Epoch 4/15
38/38 [==============================] - 3s 75ms/step - loss: 2.6880 - accuracy: 0.6922 - val_loss: 2.6869 - val_accuracy: 0.7211
Epoch 5/15
38/38 [==============================] - 3s 75ms/step - loss: 2.5981 - accuracy: 0.6928 - val_loss: 2.5636 - val_accuracy: 0.7211
Epoch 6/15
38/38 [==============================] - 3s 76ms/step - loss: 2.3889 - accuracy: 0.6928 - val_loss: 2.3292 - val_accuracy: 0.7211
Epoch 7/15
38/38 [==============================] - 3s 74ms/step - loss: 2.2007 - accuracy: 0.6928 - val_loss: 2.2198 - val_accuracy: 0.7211
Epoch 8/15
3

In [4]:
train(2)

Epoch 1/15
38/38 [==============================] - 4s 89ms/step - loss: 2.0130 - accuracy: 0.7396 - val_loss: 1.9792 - val_accuracy: 0.7434
Epoch 2/15
38/38 [==============================] - 3s 74ms/step - loss: 1.8475 - accuracy: 0.7469 - val_loss: 1.9464 - val_accuracy: 0.7427
Epoch 3/15
38/38 [==============================] - 3s 75ms/step - loss: 1.7622 - accuracy: 0.7493 - val_loss: 1.9302 - val_accuracy: 0.7484
Epoch 4/15
38/38 [==============================] - 3s 75ms/step - loss: 1.7001 - accuracy: 0.7501 - val_loss: 1.9231 - val_accuracy: 0.7505
Epoch 5/15
38/38 [==============================] - 3s 76ms/step - loss: 1.6494 - accuracy: 0.7537 - val_loss: 1.9175 - val_accuracy: 0.7537
Epoch 6/15
38/38 [==============================] - 3s 75ms/step - loss: 1.6049 - accuracy: 0.7570 - val_loss: 1.9094 - val_accuracy: 0.7550
Epoch 7/15
38/38 [==============================] - 3s 73ms/step - loss: 1.5644 - accuracy: 0.7587 - val_loss: 1.9038 - val_accuracy: 0.7606
Epoch 8/15
38

In [5]:
train(3)

Epoch 1/15
38/38 [==============================] - 4s 87ms/step - loss: 1.7097 - accuracy: 0.7953 - val_loss: 1.6075 - val_accuracy: 0.8048
Epoch 2/15
38/38 [==============================] - 3s 75ms/step - loss: 1.5325 - accuracy: 0.7998 - val_loss: 1.5830 - val_accuracy: 0.8065
Epoch 3/15
38/38 [==============================] - 3s 75ms/step - loss: 1.4399 - accuracy: 0.8024 - val_loss: 1.5691 - val_accuracy: 0.8095
Epoch 4/15
38/38 [==============================] - 3s 75ms/step - loss: 1.3666 - accuracy: 0.8079 - val_loss: 1.5640 - val_accuracy: 0.8112
Epoch 5/15
38/38 [==============================] - 3s 76ms/step - loss: 1.3051 - accuracy: 0.8125 - val_loss: 1.5603 - val_accuracy: 0.8137
Epoch 6/15
38/38 [==============================] - 3s 75ms/step - loss: 1.2498 - accuracy: 0.8151 - val_loss: 1.5555 - val_accuracy: 0.8187
Epoch 7/15
38/38 [==============================] - 3s 76ms/step - loss: 1.2002 - accuracy: 0.8195 - val_loss: 1.5529 - val_accuracy: 0.8196
Epoch 8/15
38

In [6]:
train(4)

Epoch 1/15
38/38 [==============================] - 5s 102ms/step - loss: 1.4913 - accuracy: 0.8264 - val_loss: 1.3243 - val_accuracy: 0.8421
Epoch 2/15
38/38 [==============================] - 3s 74ms/step - loss: 1.3007 - accuracy: 0.8358 - val_loss: 1.3026 - val_accuracy: 0.8482
Epoch 3/15
38/38 [==============================] - 3s 76ms/step - loss: 1.1994 - accuracy: 0.8410 - val_loss: 1.2952 - val_accuracy: 0.8495
Epoch 4/15
38/38 [==============================] - 3s 79ms/step - loss: 1.1174 - accuracy: 0.8467 - val_loss: 1.2951 - val_accuracy: 0.8520
Epoch 5/15
38/38 [==============================] - 3s 76ms/step - loss: 1.0439 - accuracy: 0.8515 - val_loss: 1.2982 - val_accuracy: 0.8537
Epoch 6/15
38/38 [==============================] - 3s 76ms/step - loss: 0.9798 - accuracy: 0.8559 - val_loss: 1.2984 - val_accuracy: 0.8558
Epoch 7/15
38/38 [==============================] - 3s 77ms/step - loss: 0.9206 - accuracy: 0.8605 - val_loss: 1.3006 - val_accuracy: 0.8583
Epoch 8/15
3

In [7]:
train(1)

Epoch 1/15
38/38 [==============================] - 4s 90ms/step - loss: 1.1745 - accuracy: 0.8598 - val_loss: 1.2714 - val_accuracy: 0.8657
Epoch 2/15
38/38 [==============================] - 3s 76ms/step - loss: 1.0398 - accuracy: 0.8672 - val_loss: 1.2575 - val_accuracy: 0.8686
Epoch 3/15
38/38 [==============================] - 3s 77ms/step - loss: 0.9607 - accuracy: 0.8732 - val_loss: 1.2548 - val_accuracy: 0.8716
Epoch 4/15
38/38 [==============================] - 3s 75ms/step - loss: 0.8995 - accuracy: 0.8779 - val_loss: 1.2595 - val_accuracy: 0.8728
Epoch 5/15
38/38 [==============================] - 3s 77ms/step - loss: 0.8465 - accuracy: 0.8821 - val_loss: 1.2607 - val_accuracy: 0.8731
Epoch 6/15
38/38 [==============================] - 3s 76ms/step - loss: 0.7977 - accuracy: 0.8868 - val_loss: 1.2651 - val_accuracy: 0.8747
Epoch 7/15
38/38 [==============================] - 3s 77ms/step - loss: 0.7513 - accuracy: 0.8908 - val_loss: 1.2671 - val_accuracy: 0.8758
Epoch 8/15
38

In [8]:
train(2)

Epoch 1/15
38/38 [==============================] - 4s 89ms/step - loss: 0.9225 - accuracy: 0.8846 - val_loss: 1.2119 - val_accuracy: 0.8851
Epoch 2/15
38/38 [==============================] - 3s 76ms/step - loss: 0.7723 - accuracy: 0.8922 - val_loss: 1.2206 - val_accuracy: 0.8876
Epoch 3/15
38/38 [==============================] - 3s 75ms/step - loss: 0.6982 - accuracy: 0.8995 - val_loss: 1.2174 - val_accuracy: 0.8886
Epoch 4/15
38/38 [==============================] - 3s 76ms/step - loss: 0.6441 - accuracy: 0.9057 - val_loss: 1.2257 - val_accuracy: 0.8897
Epoch 5/15
38/38 [==============================] - 3s 76ms/step - loss: 0.5984 - accuracy: 0.9109 - val_loss: 1.2237 - val_accuracy: 0.8901
Epoch 6/15
38/38 [==============================] - 3s 76ms/step - loss: 0.5569 - accuracy: 0.9169 - val_loss: 1.2289 - val_accuracy: 0.8909
Epoch 7/15
38/38 [==============================] - 3s 77ms/step - loss: 0.5189 - accuracy: 0.9253 - val_loss: 1.2316 - val_accuracy: 0.8910
Epoch 8/15
38

In [9]:
train(3)

Epoch 1/15
38/38 [==============================] - 5s 101ms/step - loss: 0.6816 - accuracy: 0.9051 - val_loss: 1.1494 - val_accuracy: 0.8945
Epoch 2/15
38/38 [==============================] - 3s 85ms/step - loss: 0.5326 - accuracy: 0.9174 - val_loss: 1.1456 - val_accuracy: 0.8966
Epoch 3/15
38/38 [==============================] - 3s 87ms/step - loss: 0.4625 - accuracy: 0.9299 - val_loss: 1.1481 - val_accuracy: 0.8960
Epoch 4/15
38/38 [==============================] - 3s 87ms/step - loss: 0.4122 - accuracy: 0.9439 - val_loss: 1.1508 - val_accuracy: 0.8966
Epoch 5/15
38/38 [==============================] - 3s 86ms/step - loss: 0.3715 - accuracy: 0.9578 - val_loss: 1.1508 - val_accuracy: 0.8966
Epoch 6/15
38/38 [==============================] - 3s 85ms/step - loss: 0.3348 - accuracy: 0.9699 - val_loss: 1.1567 - val_accuracy: 0.8975
Epoch 7/15
38/38 [==============================] - 3s 86ms/step - loss: 0.3031 - accuracy: 0.9780 - val_loss: 1.1589 - val_accuracy: 0.8985
Epoch 8/15
3

In [10]:
train(4)

Epoch 1/15
38/38 [==============================] - 4s 95ms/step - loss: 0.4680 - accuracy: 0.9314 - val_loss: 1.0638 - val_accuracy: 0.9097
Epoch 2/15
38/38 [==============================] - 3s 82ms/step - loss: 0.3414 - accuracy: 0.9678 - val_loss: 1.0678 - val_accuracy: 0.9109
Epoch 3/15
38/38 [==============================] - 3s 82ms/step - loss: 0.2772 - accuracy: 0.9809 - val_loss: 1.0735 - val_accuracy: 0.9112
Epoch 4/15
38/38 [==============================] - 3s 82ms/step - loss: 0.2318 - accuracy: 0.9886 - val_loss: 1.0813 - val_accuracy: 0.9114
Epoch 5/15
38/38 [==============================] - 3s 81ms/step - loss: 0.1973 - accuracy: 0.9927 - val_loss: 1.0847 - val_accuracy: 0.9120
Epoch 6/15
38/38 [==============================] - 3s 82ms/step - loss: 0.1698 - accuracy: 0.9946 - val_loss: 1.0903 - val_accuracy: 0.9118
Epoch 7/15
38/38 [==============================] - 3s 83ms/step - loss: 0.1475 - accuracy: 0.9966 - val_loss: 1.0931 - val_accuracy: 0.9118
Epoch 8/15
38

In [11]:
model = keras.models.load_model('model.h5')

def predict(raw):
    
    test = raw.split(" ")
    for i in range(MAX_LEN-len(test)):
        test.append("")

    test = [test]
    test = tokenize(test, wordToNum)


    pred = model.predict(test)

    normalized = []
    for i in range(MAX_LEN):
        normalized.append(numToWord[np.argmax(pred[0][i])])

    normalized = " ".join(normalized)

    return normalized

In [12]:
raw = "U r not playin wif Me"

norm = predict(raw)
print("Raw : ", raw)
print("Norm: ", norm)

Raw :  U r not playin wif Me
Norm:  you are not playing with me                                            


In [13]:
raw = "omg u r so funny , LOL"

norm = predict(raw)
print("Raw : ", raw)
print("Norm: ", norm)

Raw :  omg u r so funny , LOL
Norm:  oh my god you are so funny , laughing out loud                                           


In [14]:
raw = "tbh idk anything abt it"

norm = predict(raw)
print("Raw : ", raw)
print("Norm: ", norm)

Raw :  tbh idk anything abt it
Norm:  to be honest i don't know anything about it                                             


In [15]:
raw = "yea ... didnt Look lik it"

norm = predict(raw)
print("Raw : ", raw)
print("Norm: ", norm)

Raw :  yea ... didnt Look lik it
Norm:  yeah ... didn't look like it                                            
